# Adaptive Crop & Area Optimization (ACA-O) System
## Data Distribution, Variance & Model Generalization Analysis

**4th Year Project - Sri Lanka Agricultural Optimization**

This notebook provides comprehensive analysis of multi-source agricultural data for building an intelligent crop optimization system.

---

### Objectives
1. **Data Distribution Analysis** - Statistical and visual exploration of all 6 datasets
2. **Crop Recommendation** - Region and climate-based optimal crop selection
3. **Yield Prediction** - Environmental factor-driven yield forecasting
4. **Area Optimization** - Linear programming for land allocation
5. **Price Forecasting** - LSTM-based market price prediction
6. **Model Generalization** - Cross-validation and temporal/spatial splits

---

### Datasets Overview

| # | Dataset | Type | Records | ACA-O Module | Key Features |
|---|---------|------|---------|--------------|--------------|
| 1 | **Paddy Cultivation Statistics** | Yield/Area | 28 districts | Yield Prediction | District-wise sown/harvested areas, yields by irrigation scheme |
| 2 | **Economy & Rice Production (1960-2020)** | Economic | 61 years | Economic Context | GDP, inflation, population, rice production trends |
| 3 | **Vegetable Price Forecasting** | Market Prices | Variable | Price Forecasting | Multi-market vegetable price data |
| 4 | **Historical Veg & Fruit Prices** | Prices + Climate | 130,000+ | Main Training Data | Region, climate, yield impact, prices across 25 districts |
| 5 | **SriOryzia Rice Time Series** | Time Series | 324 months | Price LSTM | Monthly rice prices, production, exchange rates (1996-2022) |
| 6 | **Climate & Meteorology** | Weather | Variable | Climate Risk | Temperature, rainfall, humidity patterns |

## 1. Configuration & Imports

Consolidated imports and global settings for reproducibility.

In [ ]:
# =============================================================================
# IMPORTS & GLOBAL CONFIGURATION
# =============================================================================

import os
import warnings
from pathlib import Path
import zipfile
import glob
import math
import json
import pickle

# Data & Numerical
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec

# Machine Learning - sklearn
from sklearn.model_selection import train_test_split, KFold, cross_val_score, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# Deep Learning - PyTorch
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# =============================================================================
# GLOBAL SETTINGS
# =============================================================================

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# Visualization settings
plt.style.use('default')
sns.set_theme(style='whitegrid', palette='husl')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['axes.labelsize'] = 10

# Device configuration
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"Random Seed: {RANDOM_SEED}")
print(f"Device: {DEVICE}")
print(f"PyTorch: {torch.__version__}")
print(f"NumPy: {np.__version__}")
print(f"Pandas: {pd.__version__}")

## 2. Data Acquisition & PDF Extraction

This section downloads all Sri Lanka agricultural datasets from:
1. **Kaggle** - CSV datasets (paddy, climate, prices, economy)
2. **Official Government PDFs** - DCS, HARTI, CBSL, Irrigation Dept
3. **Direct PDF URLs** - Hydrological reports, agro-ecological maps

**Requirements:**
- Kaggle API: Place `kaggle.json` in `~/.kaggle/` directory
- Java JRE/JDK: Required for tabula-py PDF extraction

In [ ]:
# =============================================================================
# 2.1 Install Dependencies & Setup Directories
# =============================================================================

import subprocess
import sys

# Install required packages (run once)
subprocess.check_call([
    sys.executable, "-m", "pip", "install", "-q",
    "kaggle", "requests", "beautifulsoup4", "tabula-py", "pdfplumber"
])

import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import zipfile

# Directory layout
BASE_DIR      = Path("data")
KAGGLE_DIR    = BASE_DIR / "kaggle"
PDF_DIR       = BASE_DIR / "pdf"
EXTRACTED_DIR = BASE_DIR / "pdf_extracted"
PROCESSED_DIR = BASE_DIR / "processed"
RAW_DIR       = BASE_DIR / "raw"
MODELS_DIR    = Path("../app/models")

for d in [BASE_DIR, KAGGLE_DIR, PDF_DIR, EXTRACTED_DIR, PROCESSED_DIR, RAW_DIR, MODELS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print(f"Data Directory Structure:")
print(f"  Base:      {BASE_DIR.resolve()}")
print(f"  Kaggle:    {KAGGLE_DIR.resolve()}")
print(f"  PDFs:      {PDF_DIR.resolve()}")
print(f"  Extracted: {EXTRACTED_DIR.resolve()}")
print(f"  Processed: {PROCESSED_DIR.resolve()}")
print(f"  Models:    {MODELS_DIR.resolve()}")

### 2.2 Dataset Configuration

**Kaggle Datasets (Sri Lanka Only):**
| Dataset | Description |
|---------|-------------|
| paddy_cultivation_sl | Paddy cultivation statistics by district |
| rice_time_series_sl | Multivariate rice price forecasting data |
| veg_fruit_prices_sl | Historical vegetable and fruit prices |
| veg_price_forecasting_sl | Vegetable price forecasting data |
| weather_sl | Sri Lanka weather dataset |
| economy_sl | Economy of Sri Lanka indicators |
| climate_data_sl | Sri Lanka climate data |
| economy_rice_sl | Economy and rice production 1960-2020 |

**Official PDF Sources:**
- DCS Paddy Statistics (all seasons)
- HARTI Daily & Monthly Food Commodities Bulletins
- CBSL Agriculture Data Bulletin
- Irrigation Dept Reservoirs Status
- DOA Soil Fertility Maps
- Hydrological Annual Reports

In [ ]:
# =============================================================================
# 2.2.1 Kaggle Datasets Configuration
# =============================================================================

KAGGLE_DATASETS = {
    # Paddy & Rice
    "paddy_cultivation_sl":     "tharindumadhusanka9/paddy-cultivation-statics-in-sri-lanka",
    "rice_time_series_sl":      "luqmanrumaiz/srioryzia-multivariate-rice-price-forecasting",
    "economy_rice_sl":          "namalrathnayake1990/economy-and-rice-production-sri-lanka-1960-2020",
    
    # Vegetables & Fruits
    "veg_fruit_prices_sl":      "isuranga/historical-vegetable-and-fruit-prices-in-sri-lanka",
    "veg_price_forecasting_sl": "nisith210144g/vegi-price",
    
    # Climate & Weather
    "weather_sl":               "rasulmah/sri-lanka-weather-dataset",
    "climate_data_sl":          "tharindumadhusanka9/sri-lanka-climate-data",
    
    # Economy
    "economy_sl":               "amritharj/economy-of-sri-lanka",
}

# =============================================================================
# 2.2.2 Official PDF Pages (Government Sources)
# =============================================================================

OFFICIAL_PDF_PAGES = {
    # Department of Census & Statistics – Paddy Statistics
    "dcs_paddy_statistics":
        "https://www.statistics.gov.lk/Agriculture/StaticalInformation/Paddy_Statistics",

    # HARTI price bulletins – daily & monthly food commodities
    "harti_daily":
        "https://www.harti.gov.lk/index.php/en/market-information/data-food-commodities-bulletin",
    "harti_monthly":
        "https://www.harti.gov.lk/index.php/en/market-information/monthly-food-commodities-bulletin",

    # Central Bank – Agriculture Data Bulletin
    "cbsl_agri_bulletin":
        "https://www.cbsl.gov.lk/en/agriculture-data-bulletin",

    # Irrigation Dept – reservoirs status
    "irrigation_reservoirs":
        "https://irrigation.gov.lk/web/index.php?Itemid=101&catid=33&id=84%3Areservoirs&lang=en&option=com_content&view=article",

    # DOA – soil fertility maps
    "doa_soil_fertility":
        "https://doa.gov.lk/soil-fertility-map/",

    # NSDI – thematic maps
    "nsdi_thematic_maps":
        "https://nsdi.gov.lk/thematic-maps",
}

# =============================================================================
# 2.2.3 Direct PDF URLs (Known Important Documents)
# =============================================================================

DIRECT_PDF_URLS = {
    "irrigation_hydrological_annual_2018_19":
        "https://www.irrigation.gov.lk/web/images/Hydrological-Annual/12_Hydrological_Annual_18-19.pdf",
    "irrigation_hydrological_annual_2020_21":
        "https://www.irrigation.gov.lk/web/images/Publications/Hydrologyical_Annual_2020-21.pdf",
    "agro_ecological_regions_jrc":
        "https://esdac.jrc.ec.europa.eu/images/Eudasm/Asia/images/maps/download/PDF/LK2007_CL.pdf",
}

print(f"Configured {len(KAGGLE_DATASETS)} Kaggle datasets")
print(f"Configured {len(OFFICIAL_PDF_PAGES)} official PDF pages")
print(f"Configured {len(DIRECT_PDF_URLS)} direct PDF URLs")

In [ ]:
# =============================================================================
# 2.3 Download Functions
# =============================================================================

def download_kaggle_datasets(kaggle_map: dict, out_dir: Path):
    """
    Download and unzip all Kaggle datasets into out_dir/<nickname>
    Requires: ~/.kaggle/kaggle.json
    """
    print("\n" + "=" * 60)
    print("DOWNLOADING KAGGLE DATASETS")
    print("=" * 60)
    
    for nickname, slug in kaggle_map.items():
        target = out_dir / nickname
        target.mkdir(parents=True, exist_ok=True)
        
        # Skip if already downloaded
        if list(target.glob('*.csv')):
            print(f"✓ {nickname}: Already exists")
            continue
            
        print(f"\n→ {nickname}: {slug}")
        try:
            subprocess.run(
                ["kaggle", "datasets", "download", "-d", slug, "-p", str(target), "--unzip"],
                check=True,
                capture_output=True
            )
            print(f"  ✓ Downloaded successfully")
        except subprocess.CalledProcessError as e:
            print(f"  ✗ Failed: {e}")
        except FileNotFoundError:
            print(f"  ✗ Kaggle CLI not found. Install with: pip install kaggle")
            break
    
    print("\n[Kaggle] Finished downloading Sri Lanka datasets.")


def download_pdfs_from_page(name: str, url: str, base_out_dir: Path, timeout: int = 40):
    """
    Fetch a web page and download ALL linked .pdf files.
    """
    print(f"\n{'=' * 60}")
    print(f"SCRAPING: {name}")
    print(f"{'=' * 60}")
    print(f"URL: {url}")

    out_dir = base_out_dir / name
    out_dir.mkdir(parents=True, exist_ok=True)

    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
        resp = requests.get(url, timeout=timeout, headers=headers)
        resp.raise_for_status()
    except Exception as e:
        print(f"✗ Failed to fetch page: {e}")
        return

    soup = BeautifulSoup(resp.text, "html.parser")
    links = soup.find_all("a", href=True)
    pdf_links = []

    for a in links:
        href = a["href"]
        if ".pdf" in href.lower():
            full_url = urljoin(url, href)
            pdf_links.append(full_url)

    pdf_links = sorted(set(pdf_links))
    
    if not pdf_links:
        print("  (No PDF links found on this page)")
        return

    print(f"  Found {len(pdf_links)} PDF file(s)")
    
    for pdf_url in pdf_links:
        filename = pdf_url.split("/")[-1].split("?")[0]
        # Clean filename
        filename = filename.replace("%20", "_").replace(" ", "_")
        dest = out_dir / filename
        
        if dest.exists():
            print(f"  ✓ {filename}: Already exists")
            continue
            
        try:
            print(f"  → Downloading: {filename}")
            with requests.get(pdf_url, stream=True, timeout=timeout, headers=headers) as r:
                r.raise_for_status()
                with open(dest, "wb") as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
            print(f"    ✓ Saved")
        except Exception as e:
            print(f"    ✗ Failed: {e}")


def download_direct_pdfs(pdf_map: dict, base_out_dir: Path, timeout: int = 40):
    """
    Download direct PDF URLs into base_out_dir/direct_pdfs
    """
    print("\n" + "=" * 60)
    print("DOWNLOADING DIRECT PDF URLS")
    print("=" * 60)
    
    misc_dir = base_out_dir / "direct_pdfs"
    misc_dir.mkdir(parents=True, exist_ok=True)

    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
    
    for name, url in pdf_map.items():
        filename = f"{name}.pdf"
        dest = misc_dir / filename
        
        if dest.exists():
            print(f"✓ {name}: Already exists")
            continue
            
        try:
            print(f"→ Downloading: {name}")
            with requests.get(url, stream=True, timeout=timeout, headers=headers) as r:
                r.raise_for_status()
                with open(dest, "wb") as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
            print(f"  ✓ Saved")
        except Exception as e:
            print(f"  ✗ Failed: {e}")


def extract_tables_from_pdfs(pdf_root: Path, out_root: Path):
    """
    Extract ALL tables from PDFs into CSV files using tabula-py.
    Requires Java JRE/JDK installed.
    """
    print("\n" + "=" * 60)
    print("EXTRACTING TABLES FROM PDFs → CSV")
    print("=" * 60)
    
    try:
        import tabula
    except ImportError:
        print("✗ tabula-py not installed. Run: pip install tabula-py")
        return
    
    pdf_root = Path(pdf_root)
    out_root = Path(out_root)
    out_root.mkdir(parents=True, exist_ok=True)

    pdf_files = list(pdf_root.rglob("*.pdf"))
    
    if not pdf_files:
        print("  (No PDFs found to extract)")
        return

    print(f"  Found {len(pdf_files)} PDF file(s)")
    
    for pdf_path in pdf_files:
        rel_dir = pdf_path.parent.relative_to(pdf_root)
        target_dir = out_root / rel_dir
        target_dir.mkdir(parents=True, exist_ok=True)

        print(f"\n  → Processing: {pdf_path.name}")
        
        try:
            tables = tabula.read_pdf(
                str(pdf_path),
                pages="all",
                multiple_tables=True,
                silent=True
            )
        except Exception as e:
            print(f"    ✗ Failed to read: {e}")
            continue

        if not tables:
            print("    (No tables detected)")
            continue

        print(f"    Found {len(tables)} table(s)")
        
        for i, df in enumerate(tables, start=1):
            if df.empty:
                continue
            clean_stem = pdf_path.stem.replace(" ", "_").replace("-", "_")
            out_csv = target_dir / f"{clean_stem}_table{i}.csv"
            try:
                df.to_csv(out_csv, index=False)
                print(f"    ✓ Saved: {out_csv.name}")
            except Exception as e:
                print(f"    ✗ Failed to save table {i}: {e}")

print("Download functions defined successfully")

In [ ]:
# =============================================================================
# 2.4 Execute Data Download Pipeline
# =============================================================================

def run_full_data_pipeline():
    """
    Main function to download all datasets and extract PDFs.
    """
    print("=" * 70)
    print("  ACA-O DATA ACQUISITION PIPELINE")
    print("  Sri Lanka Agricultural Optimization Datasets")
    print("=" * 70)
    
    # 1) Download Kaggle CSV datasets
    download_kaggle_datasets(KAGGLE_DATASETS, KAGGLE_DIR)
    
    # 2) Scrape and download PDFs from official government pages
    for name, url in OFFICIAL_PDF_PAGES.items():
        download_pdfs_from_page(name, url, PDF_DIR)
    
    # 3) Download direct PDF URLs (hydrology reports, agro-ecological maps)
    download_direct_pdfs(DIRECT_PDF_URLS, PDF_DIR)
    
    # 4) Extract tables from ALL downloaded PDFs → CSV
    extract_tables_from_pdfs(PDF_DIR, EXTRACTED_DIR)
    
    # Summary
    print("\n" + "=" * 70)
    print("  PIPELINE COMPLETE")
    print("=" * 70)
    
    # Count files
    kaggle_csvs = list(KAGGLE_DIR.rglob("*.csv"))
    pdf_files = list(PDF_DIR.rglob("*.pdf"))
    extracted_csvs = list(EXTRACTED_DIR.rglob("*.csv"))
    
    print(f"\n📁 Kaggle CSVs:     {len(kaggle_csvs)} files in {KAGGLE_DIR}")
    print(f"📄 Downloaded PDFs: {len(pdf_files)} files in {PDF_DIR}")
    print(f"📊 Extracted CSVs:  {len(extracted_csvs)} files in {EXTRACTED_DIR}")
    
    return {
        'kaggle_csvs': kaggle_csvs,
        'pdf_files': pdf_files,
        'extracted_csvs': extracted_csvs
    }

# Run the full pipeline
pipeline_results = run_full_data_pipeline()

## 2.3 Data Loading

Load all downloaded datasets from Kaggle and extracted PDF tables into DataFrames.

In [ ]:
# =============================================================================
# 3.1 CSV Loading Utilities
# =============================================================================

def load_csv_from_folder(folder: Path, prefer_largest: bool = True) -> pd.DataFrame:
    """
    Load CSV file(s) from a folder.
    If multiple CSVs exist, loads the largest one by default.
    """
    csv_files = list(folder.glob('*.csv')) or list(folder.rglob('*.csv'))
    
    if not csv_files:
        raise FileNotFoundError(f'No CSV files found in {folder}')
    
    if prefer_largest:
        main_csv = max(csv_files, key=lambda p: p.stat().st_size)
    else:
        main_csv = csv_files[0]
    
    # Try different encodings
    for encoding in ['utf-8', 'latin-1', 'cp1252', 'iso-8859-1']:
        try:
            df = pd.read_csv(main_csv, encoding=encoding, low_memory=False)
            print(f"  ✓ Loaded {main_csv.name}: {df.shape[0]} rows × {df.shape[1]} cols")
            return df
        except UnicodeDecodeError:
            continue
    
    # Fallback with error replacement
    df = pd.read_csv(main_csv, encoding='utf-8', errors='replace', low_memory=False)
    print(f"  ✓ Loaded {main_csv.name} (with encoding fallback): {df.shape}")
    return df


def load_all_csvs_from_folder(folder: Path) -> dict:
    """
    Load ALL CSV files from a folder into a dictionary.
    """
    csv_files = list(folder.rglob('*.csv'))
    results = {}
    
    for csv_file in csv_files:
        try:
            df = pd.read_csv(csv_file, encoding='utf-8', low_memory=False, errors='replace')
            key = csv_file.stem
            results[key] = df
        except Exception as e:
            print(f"  ✗ Failed to load {csv_file.name}: {e}")
    
    return results

print("CSV loading utilities defined")

In [ ]:
# =============================================================================
# 3.2 Load All Kaggle Datasets
# =============================================================================

print("Loading Kaggle Datasets...")
print("=" * 60)

# Primary datasets
df_paddy = load_csv_from_folder(KAGGLE_DIR / 'paddy_cultivation_sl')
df_rice_ts = load_csv_from_folder(KAGGLE_DIR / 'rice_time_series_sl')
df_economy_rice = load_csv_from_folder(KAGGLE_DIR / 'economy_rice_sl')
df_hist_veg = load_csv_from_folder(KAGGLE_DIR / 'veg_fruit_prices_sl')
df_veg_price = load_csv_from_folder(KAGGLE_DIR / 'veg_price_forecasting_sl')
df_weather = load_csv_from_folder(KAGGLE_DIR / 'weather_sl')
df_climate = load_csv_from_folder(KAGGLE_DIR / 'climate_data_sl')
df_economy = load_csv_from_folder(KAGGLE_DIR / 'economy_sl')

# Store all in a dictionary for easy access
DATASETS = {
    'paddy': df_paddy,
    'rice_ts': df_rice_ts,
    'economy_rice': df_economy_rice,
    'hist_veg': df_hist_veg,
    'veg_price': df_veg_price,
    'weather': df_weather,
    'climate': df_climate,
    'economy': df_economy,
}

print("\n" + "=" * 60)
print("KAGGLE DATASETS SUMMARY")
print("=" * 60)
for name, df in DATASETS.items():
    print(f"  {name:15} : {df.shape[0]:>7,} rows × {df.shape[1]:>3} cols")
print(f"\n  Total: {sum(df.shape[0] for df in DATASETS.values()):,} records")

In [ ]:
# =============================================================================
# 3.3 Load Extracted PDF Tables
# =============================================================================

print("Loading Extracted PDF Tables...")
print("=" * 60)

# Load all extracted CSVs from PDFs
pdf_tables = load_all_csvs_from_folder(EXTRACTED_DIR)

print(f"\nFound {len(pdf_tables)} extracted tables from PDFs")

# Categorize by source
dcs_tables = {k: v for k, v in pdf_tables.items() if 'dcs' in k.lower() or 'paddy' in k.lower()}
harti_tables = {k: v for k, v in pdf_tables.items() if 'harti' in k.lower()}
irrigation_tables = {k: v for k, v in pdf_tables.items() if 'irrigation' in k.lower() or 'hydro' in k.lower()}
other_tables = {k: v for k, v in pdf_tables.items() 
                if k not in dcs_tables and k not in harti_tables and k not in irrigation_tables}

print(f"\nBy Category:")
print(f"  DCS (Paddy/Census):  {len(dcs_tables)} tables")
print(f"  HARTI (Prices):      {len(harti_tables)} tables")
print(f"  Irrigation:          {len(irrigation_tables)} tables")
print(f"  Other:               {len(other_tables)} tables")

# Preview extracted tables
if pdf_tables:
    print("\n" + "-" * 60)
    print("Sample Extracted Tables:")
    for name, df in list(pdf_tables.items())[:5]:
        print(f"  • {name}: {df.shape}")

## 3. Exploratory Data Analysis

Quick exploration of loaded datasets to understand their structure and contents.

In [ ]:
def eda(df: pd.DataFrame, name: str):
    print(f'\n{name}: {df.shape[0]} rows, {df.shape[1]} cols')
    print(f'Columns: {list(df.columns)}')
    display(df.head(3))

eda(df_paddy, 'Paddy Stats')
eda(df_economy, 'Economy & Rice')
eda(df_hist_veg, 'Historical Prices')
eda(df_rice_ts, 'Rice Time Series')

## 4. Data Distribution & Coverage Analysis

This section provides statistical analysis of data distributions across all datasets, identifying:
- Feature distributions and skewness
- Temporal coverage and gaps
- Regional representation
- Class imbalances in categorical variables

In [ ]:
# =============================================================================
# 3.1 Dataset Statistics Summary
# =============================================================================

def dataset_summary(datasets_dict):
    """Generate statistical summary for all datasets."""
    summary_data = []
    
    for name, df in datasets_dict.items():
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        summary_data.append({
            'Dataset': name,
            'Records': len(df),
            'Features': len(df.columns),
            'Numeric': len(numeric_cols),
            'Categorical': len(df.columns) - len(numeric_cols),
            'Missing (%)': round(df.isnull().sum().sum() / (df.shape[0] * df.shape[1]) * 100, 2),
            'Memory (KB)': round(df.memory_usage(deep=True).sum() / 1024, 1)
        })
    
    return pd.DataFrame(summary_data)

# Create datasets dictionary
datasets = {
    'Paddy Statistics': df_paddy,
    'Economy (Rice)': df_economy,
    'Vegetable Prices': df_veg_price,
    'Historical Vegetables': df_hist_veg,
    'Rice Time Series': df_rice_ts,
    'Climate Data': df_climate
}

summary_table = dataset_summary(datasets)
print("DATASET STATISTICS OVERVIEW")
print("=" * 80)
print(summary_table.to_string(index=False))
print(f"\nTotal Records: {summary_table['Records'].sum():,}")
print(f"Total Memory: {summary_table['Memory (KB)'].sum():,.1f} KB")

In [ ]:
# =============================================================================
# 3.2 Feature Distribution Analysis
# =============================================================================

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Feature Distribution Analysis Across Datasets', fontsize=14, fontweight='bold')

# 1. Paddy - use numeric columns only
ax = axes[0, 0]
paddy_numeric = df_paddy.select_dtypes(include=[np.number])
if len(paddy_numeric.columns) > 0:
    first_col = paddy_numeric.columns[0]
    data = pd.to_numeric(paddy_numeric[first_col], errors='coerce').dropna()
    if len(data) > 0:
        data.hist(ax=ax, bins=15, edgecolor='black', alpha=0.7)
        ax.axvline(data.mean(), color='red', linestyle='--', label=f'Mean: {data.mean():.0f}')
        ax.set_title(f'Paddy: {first_col[:20]}')
        ax.legend(fontsize=8)

# 2. Rice production over time
ax = axes[0, 1]
rice_prod_col = [c for c in df_economy.columns if 'rice' in c.lower() and 'production' in c.lower()]
if rice_prod_col and 'Year' in df_economy.columns:
    df_economy.plot(x='Year', y=rice_prod_col[0], ax=ax, marker='o', legend=False)
    ax.set_title('Rice Production Trend')
    ax.set_ylabel('Production (Mt)')
else:
    econ_numeric = df_economy.select_dtypes(include=[np.number])
    if len(econ_numeric.columns) > 1:
        ax.plot(econ_numeric.iloc[:, 0], econ_numeric.iloc[:, 1], marker='o')
        ax.set_title('Economic Trend')

# 3. Historical vegetable price distribution  
ax = axes[0, 2]
hist_numeric = df_hist_veg.select_dtypes(include=[np.number])
if len(hist_numeric.columns) > 0:
    price_cols = [c for c in hist_numeric.columns if 'price' in c.lower() or 'avg' in c.lower()]
    if price_cols:
        data = pd.to_numeric(hist_numeric[price_cols[0]], errors='coerce').dropna()
    else:
        data = pd.to_numeric(hist_numeric.iloc[:, 0], errors='coerce').dropna()
    if len(data) > 0:
        data.hist(ax=ax, bins=50, edgecolor='black', alpha=0.7, log=True)
        ax.set_title('Vegetable Price Distribution (Log)')

# 4. Historical vegetable volume distribution
ax = axes[1, 0]
vol_cols = [c for c in hist_numeric.columns if 'volume' in c.lower() or 'qty' in c.lower()]
if vol_cols:
    data = pd.to_numeric(hist_numeric[vol_cols[0]], errors='coerce').dropna()
    if len(data) > 0:
        data.hist(ax=ax, bins=50, edgecolor='black', alpha=0.7, log=True)
        ax.set_title('Trading Volume (Log Scale)')

# 5. Rice time series - price trend
ax = axes[1, 1]
rice_numeric = df_rice_ts.select_dtypes(include=[np.number])
if len(rice_numeric.columns) > 0:
    for i, col in enumerate(rice_numeric.columns[:3]):
        data = pd.to_numeric(rice_numeric[col], errors='coerce').dropna()
        ax.plot(range(len(data)), data, label=col[:15], alpha=0.7)
    ax.set_title('Rice Price Time Series')
    ax.legend(fontsize=7)

# 6. Climate feature correlation heatmap
ax = axes[1, 2]
climate_numeric = df_climate.select_dtypes(include=[np.number])
if len(climate_numeric.columns) > 1:
    # Convert to numeric and compute correlation
    climate_clean = climate_numeric.apply(pd.to_numeric, errors='coerce').dropna()
    if len(climate_clean) > 0:
        corr_climate = climate_clean.corr()
        sns.heatmap(corr_climate, ax=ax, cmap='RdBu_r', center=0, annot=True, fmt='.2f', 
                    annot_kws={'size': 7}, cbar_kws={'shrink': 0.8})
        ax.set_title('Climate Features Correlation')

plt.tight_layout()
plt.show()

print("\nDistribution Statistics:")
print("-" * 50)
for name, df in datasets.items():
    numeric = df.select_dtypes(include=[np.number]).apply(pd.to_numeric, errors='coerce')
    if len(numeric.columns) > 0:
        skew = numeric.skew().abs().mean()
        if not np.isnan(skew):
            print(f"{name}: Mean Skewness = {skew:.3f} {'(High)' if skew > 1 else '(Normal)'}")

In [ ]:
# =============================================================================
# 3.3 Regional & Temporal Coverage Analysis
# =============================================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Data Coverage Analysis: Regional & Temporal', fontsize=14, fontweight='bold')

# 1. Regional coverage in paddy data
ax = axes[0, 0]
region_col = [c for c in df_paddy.columns if 'region' in c.lower() or 'district' in c.lower()]
if region_col:
    region_counts = df_paddy[region_col[0]].value_counts()
    region_counts.plot(kind='barh', ax=ax, color=plt.cm.viridis(np.linspace(0, 1, len(region_counts))))
    ax.set_xlabel('Number of Records')
    ax.set_title(f'Regional Coverage: Paddy Data ({len(region_counts)} regions)')
    ax.axvline(region_counts.mean(), color='red', linestyle='--', label=f'Mean: {region_counts.mean():.0f}')

# 2. Temporal coverage
ax = axes[0, 1]
temporal_data = []

# Check each dataset for temporal columns
for name, df in datasets.items():
    year_cols = [c for c in df.columns if 'year' in c.lower()]
    date_cols = [c for c in df.columns if 'date' in c.lower()]
    
    if year_cols:
        years = pd.to_numeric(df[year_cols[0]], errors='coerce').dropna()
        if len(years) > 0:
            temporal_data.append({
                'Dataset': name[:15],
                'Min Year': int(years.min()),
                'Max Year': int(years.max()),
                'Span': int(years.max() - years.min())
            })

if temporal_data:
    temp_df = pd.DataFrame(temporal_data)
    y_pos = range(len(temp_df))
    ax.barh(y_pos, temp_df['Span'], left=temp_df['Min Year'], height=0.6, alpha=0.7)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(temp_df['Dataset'])
    ax.set_xlabel('Year')
    ax.set_title('Temporal Coverage by Dataset')
    for i, row in temp_df.iterrows():
        ax.text(row['Min Year'] + row['Span']/2, i, f"{row['Min Year']}-{row['Max Year']}", 
                ha='center', va='center', fontsize=8, fontweight='bold')

# 3. Crop distribution in historical data
ax = axes[1, 0]
crop_col = [c for c in df_hist_veg.columns if 'crop' in c.lower() or 'commodity' in c.lower() or 'item' in c.lower()]
if crop_col:
    crop_counts = df_hist_veg[crop_col[0]].value_counts().head(15)
    colors = plt.cm.Set3(np.linspace(0, 1, len(crop_counts)))
    crop_counts.plot(kind='bar', ax=ax, color=colors, edgecolor='black')
    ax.set_title(f'Top 15 Crops by Record Count (Total: {df_hist_veg[crop_col[0]].nunique()})')
    ax.set_xlabel('Crop')
    ax.set_ylabel('Records')
    ax.tick_params(axis='x', rotation=45)
    
    # Calculate imbalance ratio
    imbalance = crop_counts.max() / crop_counts.min()
    ax.text(0.95, 0.95, f'Imbalance Ratio: {imbalance:.1f}x', transform=ax.transAxes,
            fontsize=9, ha='right', va='top', bbox=dict(boxstyle='round', facecolor='wheat'))

# 4. Missing data heatmap
ax = axes[1, 1]
missing_pct = pd.DataFrame({name: (df.isnull().sum() / len(df) * 100).head(10) 
                            for name, df in datasets.items()})
if missing_pct.sum().sum() > 0:
    sns.heatmap(missing_pct.T, ax=ax, cmap='YlOrRd', annot=True, fmt='.1f',
                cbar_kws={'label': 'Missing %'})
    ax.set_title('Missing Data Analysis (Top 10 Features)')
else:
    ax.text(0.5, 0.5, 'No Missing Data\nAll datasets complete', ha='center', va='center',
            fontsize=14, transform=ax.transAxes)
    ax.set_title('Missing Data Analysis')

plt.tight_layout()
plt.show()

In [ ]:
# =============================================================================
# 3.4 Feature Variance & Outlier Analysis
# =============================================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Feature Variance & Outlier Detection', fontsize=14, fontweight='bold')

# 1. Coefficient of Variation across datasets
ax = axes[0, 0]
cv_data = {}
for name, df in datasets.items():
    numeric = df.select_dtypes(include=[np.number])
    if len(numeric.columns) > 0:
        cv = (numeric.std() / numeric.mean().replace(0, np.nan)).dropna()
        cv_data[name[:12]] = cv.mean()

if cv_data:
    bars = ax.bar(cv_data.keys(), cv_data.values(), color=plt.cm.coolwarm(np.linspace(0.3, 0.7, len(cv_data))))
    ax.set_ylabel('Coefficient of Variation')
    ax.set_title('Data Variability by Dataset')
    ax.tick_params(axis='x', rotation=45)
    ax.axhline(y=1.0, color='red', linestyle='--', label='CV=1 threshold')
    ax.legend()

# 2. Outlier detection using IQR method
ax = axes[0, 1]
outlier_pct = {}
for name, df in datasets.items():
    numeric = df.select_dtypes(include=[np.number])
    total_outliers = 0
    total_values = 0
    for col in numeric.columns:
        Q1, Q3 = numeric[col].quantile([0.25, 0.75])
        IQR = Q3 - Q1
        outliers = ((numeric[col] < Q1 - 1.5*IQR) | (numeric[col] > Q3 + 1.5*IQR)).sum()
        total_outliers += outliers
        total_values += len(numeric[col].dropna())
    if total_values > 0:
        outlier_pct[name[:12]] = total_outliers / total_values * 100

if outlier_pct:
    colors = ['#ff6b6b' if v > 5 else '#4ecdc4' for v in outlier_pct.values()]
    ax.barh(list(outlier_pct.keys()), list(outlier_pct.values()), color=colors)
    ax.set_xlabel('Outlier Percentage (%)')
    ax.set_title('Outlier Prevalence (IQR Method)')
    ax.axvline(x=5, color='red', linestyle='--', label='5% threshold')
    ax.legend()

# 3. Box plots for key features
ax = axes[1, 0]
key_features = []
if 'Yield_kg_per_ha' in df_paddy.columns:
    key_features.append(('Paddy Yield', df_paddy['Yield_kg_per_ha']))
if 'Avg_Price' in df_hist_veg.columns:
    key_features.append(('Veg Price', df_hist_veg['Avg_Price']))
elif len(df_hist_veg.select_dtypes(include=[np.number]).columns) > 0:
    price_col = df_hist_veg.select_dtypes(include=[np.number]).columns[0]
    key_features.append(('Veg Price', df_hist_veg[price_col]))

if key_features:
    box_data = [f[1].dropna().values[:1000] for f in key_features]  # Sample for performance
    bp = ax.boxplot(box_data, labels=[f[0] for f in key_features], patch_artist=True)
    colors = ['#3498db', '#e74c3c', '#2ecc71']
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
    ax.set_title('Key Feature Distributions')
    ax.set_ylabel('Value')

# 4. Skewness distribution
ax = axes[1, 1]
all_skewness = []
skew_labels = []
for name, df in datasets.items():
    numeric = df.select_dtypes(include=[np.number])
    for col in numeric.columns:
        skew_val = numeric[col].skew()
        if not np.isnan(skew_val):
            all_skewness.append(skew_val)
            skew_labels.append(f"{name[:8]}: {col[:10]}")

if all_skewness:
    sorted_idx = np.argsort(np.abs(all_skewness))[::-1][:15]  # Top 15 most skewed
    colors = ['#e74c3c' if abs(all_skewness[i]) > 1 else '#3498db' for i in sorted_idx]
    ax.barh([skew_labels[i] for i in sorted_idx], [all_skewness[i] for i in sorted_idx], color=colors)
    ax.axvline(x=0, color='black', linestyle='-')
    ax.axvline(x=-1, color='red', linestyle='--', alpha=0.5)
    ax.axvline(x=1, color='red', linestyle='--', alpha=0.5)
    ax.set_xlabel('Skewness')
    ax.set_title('Top 15 Most Skewed Features')

plt.tight_layout()
plt.show()

print("\nVariance Analysis Summary:")
print("-" * 50)
print(f"Datasets with high variability (CV > 1): {sum(1 for v in cv_data.values() if v > 1)}/{len(cv_data)}")
print(f"Datasets with >5% outliers: {sum(1 for v in outlier_pct.values() if v > 5)}/{len(outlier_pct)}")

## 5. Dataset-to-Model Mapping

This section illustrates how each dataset contributes to the ACA-O system components and their interconnections.

In [ ]:
# =============================================================================
# 4.1 Dataset-Model Relationship Visualization
# =============================================================================

fig, ax = plt.subplots(figsize=(16, 10))
ax.set_xlim(0, 100)
ax.set_ylim(0, 70)
ax.axis('off')
ax.set_title('ACA-O System: Dataset to Model Mapping', fontsize=16, fontweight='bold', pad=20)

# Define positions
datasets_pos = [(10, 60), (10, 45), (10, 30), (10, 15)]
models_pos = [(50, 55), (50, 40), (50, 25)]
output_pos = [(90, 40)]

# Draw datasets (left side)
dataset_info = [
    ('Paddy Statistics', '28 records', '#3498db'),
    ('Climate Data', '1800 records', '#2ecc71'),
    ('Historical Veg', '130K records', '#e74c3c'),
    ('Rice Time Series', '324 records', '#9b59b6')
]

for (x, y), (name, count, color) in zip(datasets_pos, dataset_info):
    rect = plt.Rectangle((x-8, y-4), 20, 8, facecolor=color, alpha=0.3, edgecolor=color, linewidth=2)
    ax.add_patch(rect)
    ax.text(x+2, y+1, name, fontsize=9, fontweight='bold', ha='center')
    ax.text(x+2, y-2, count, fontsize=8, ha='center', style='italic')

# Draw models (middle)
model_info = [
    ('CropRecommender', 'Embedding NN', '#ff6b6b'),
    ('YieldPredictor', 'MLP Regressor', '#4ecdc4'),
    ('PriceLSTM', 'Seq2Seq', '#45b7d1')
]

for (x, y), (name, arch, color) in zip(models_pos, model_info):
    ellipse = plt.Circle((x, y), 8, facecolor=color, alpha=0.3, edgecolor=color, linewidth=2)
    ax.add_patch(ellipse)
    ax.text(x, y+2, name, fontsize=9, fontweight='bold', ha='center')
    ax.text(x, y-2, arch, fontsize=8, ha='center', style='italic')

# Draw output (right side)
rect = plt.Rectangle((82, 32), 16, 16, facecolor='#f39c12', alpha=0.3, edgecolor='#f39c12', linewidth=2)
ax.add_patch(rect)
ax.text(90, 42, 'Area', fontsize=10, fontweight='bold', ha='center')
ax.text(90, 38, 'Optimizer', fontsize=10, fontweight='bold', ha='center')
ax.text(90, 34, '(Linear Prog)', fontsize=8, ha='center', style='italic')

# Draw connections
connections = [
    ((22, 60), (42, 55), '#3498db'),  # Paddy -> CropRec
    ((22, 45), (42, 55), '#2ecc71'),  # Climate -> CropRec
    ((22, 45), (42, 40), '#2ecc71'),  # Climate -> Yield
    ((22, 30), (42, 40), '#e74c3c'),  # HistVeg -> Yield
    ((22, 15), (42, 25), '#9b59b6'),  # RiceTS -> Price
    ((22, 30), (42, 25), '#e74c3c'),  # HistVeg -> Price
    ((58, 55), (82, 42), '#ff6b6b'),  # CropRec -> Optimizer
    ((58, 40), (82, 40), '#4ecdc4'),  # Yield -> Optimizer
    ((58, 25), (82, 38), '#45b7d1'),  # Price -> Optimizer
]

for (x1, y1), (x2, y2), color in connections:
    ax.annotate('', xy=(x2, y2), xytext=(x1, y1),
                arrowprops=dict(arrowstyle='->', color=color, lw=1.5, alpha=0.7))

# Add legend
ax.text(50, 5, 'Data Flow: Datasets → Models → Optimization Output', 
        fontsize=10, ha='center', style='italic', color='gray')

plt.tight_layout()
plt.show()

# Print feature mapping table
print("\nDATASET-MODEL FEATURE MAPPING")
print("=" * 80)
mapping = [
    ['Paddy Statistics', 'CropRecommender', 'Region, Season, Crop → Embedding vectors'],
    ['Climate Data', 'CropRecommender', 'Temp, Humidity, Rainfall → Climate features'],
    ['Climate Data', 'YieldPredictor', 'Climate conditions → Yield estimation'],
    ['Historical Veg', 'YieldPredictor', 'Historical yields → Training targets'],
    ['Historical Veg', 'PriceLSTM', 'Price sequences → Pattern learning'],
    ['Rice Time Series', 'PriceLSTM', 'Monthly prices → Price forecasting'],
    ['All Models', 'AreaOptimizer', 'Predictions → Linear programming constraints']
]
for src, tgt, desc in mapping:
    print(f"{src:20} → {tgt:18} | {desc}")

## 6. Economy-Rice Production Analysis

In [ ]:
# Find target column dynamically
target_candidates = [c for c in df_economy.columns if 'rice' in c.lower() and 'production' in c.lower()]
ECON_TARGET_COL = target_candidates[0] if target_candidates else df_economy.columns[-1]
print(f"Target: {ECON_TARGET_COL}")

numeric_cols = df_economy.select_dtypes(include=[np.number]).columns.tolist()
ECON_FEATURE_COLS = [c for c in numeric_cols if c != ECON_TARGET_COL]

econ_df = df_economy.dropna(subset=[ECON_TARGET_COL] + ECON_FEATURE_COLS).copy()
X_econ = econ_df[ECON_FEATURE_COLS].values.astype(np.float32)
y_econ = econ_df[ECON_TARGET_COL].values.astype(np.float32).reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X_econ, y_econ, test_size=0.2, random_state=42)

scaler_econ = StandardScaler()
X_train_scaled = scaler_econ.fit_transform(X_train)
X_test_scaled = scaler_econ.transform(X_test)

lr = LinearRegression().fit(X_train_scaled, y_train)
print(f'Linear Regression RMSE: {np.sqrt(mean_squared_error(y_test, lr.predict(X_test_scaled))):.2f}')

In [ ]:
class MLP(nn.Module):
    def __init__(self, in_features, hidden=(64, 32)):
        super().__init__()
        layers = []
        for h in hidden:
            layers.extend([nn.Linear(in_features, h), nn.ReLU()])
            in_features = h
        layers.append(nn.Linear(in_features, 1))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

mlp = MLP(len(ECON_FEATURE_COLS)).to(DEVICE)
opt = torch.optim.Adam(mlp.parameters(), lr=1e-3)
crit = nn.MSELoss()

X_t = torch.from_numpy(X_train_scaled).to(DEVICE)
y_t = torch.from_numpy(y_train).to(DEVICE)

for epoch in range(500):
    mlp.train()
    opt.zero_grad()
    loss = crit(mlp(X_t), y_t)
    loss.backward()
    opt.step()

mlp.eval()
with torch.no_grad():
    pred = mlp(torch.from_numpy(X_test_scaled).to(DEVICE)).cpu().numpy()
print(f'MLP RMSE: {np.sqrt(mean_squared_error(y_test, pred)):.2f}')

## 7. Data Preprocessing

In [ ]:
# Historical vegetable/fruit prices with climate
df_hist = df_hist_veg.rename(columns={
    'Date': 'date', 'Region': 'region',
    'Temperature (°C)': 'temperature', 'Rainfall (mm)': 'rainfall',
    'Humidity (%)': 'humidity', 'Crop Yield Impact Score': 'yield_impact_score',
    'fruit_Commodity': 'fruit_type', 'fruit_Price per Unit (LKR/kg)': 'fruit_price',
    'vegitable_Commodity': 'vegetable_type', 'vegitable_Price per Unit (LKR/kg)': 'vegetable_price'
})
df_hist['date'] = pd.to_datetime(df_hist['date'], errors='coerce')

# Convert numeric columns
for col in ['temperature', 'rainfall', 'humidity', 'yield_impact_score', 'vegetable_price', 'fruit_price']:
    df_hist[col] = pd.to_numeric(df_hist[col], errors='coerce')

# Rice time series
df_rice = df_rice_ts.copy()
df_rice['date'] = pd.to_datetime(df_rice['date'], errors='coerce')
df_rice = df_rice[['date', 'price', 'production', 'production_total', 'exchange_rate', 'fuel_price']].dropna()

print(f"Historical data: {len(df_hist)} rows, {df_hist['region'].nunique()} regions, "
      f"{df_hist['vegetable_type'].nunique()} vegetables")
print(f"Rice data: {len(df_rice)} rows, date range: {df_rice['date'].min()} to {df_rice['date'].max()}")

## 8. Climate-Yield Analysis

In [ ]:
# Climate-yield correlation
climate_cols = ['temperature', 'rainfall', 'humidity', 'yield_impact_score']
df_climate_yield = df_hist[climate_cols].dropna()

corr = df_climate_yield.corr()
print("Correlation with yield_impact_score:")
print(corr['yield_impact_score'].sort_values(ascending=False))

# Regional aggregation
regional = df_hist.groupby('region').agg({
    'temperature': 'mean', 'rainfall': 'mean', 'humidity': 'mean',
    'yield_impact_score': 'mean', 'vegetable_price': 'mean', 'fruit_price': 'mean'
}).round(2).sort_values('yield_impact_score', ascending=False)

display(regional.head(10))

## 9. Crop Recommendation Model

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Prepare crop recommendation data
df_crop = df_hist[['region', 'date', 'temperature', 'rainfall', 'humidity', 
                   'yield_impact_score', 'vegetable_type']].dropna()
df_crop['month'] = df_crop['date'].dt.month

# Encoders
region_enc = LabelEncoder()
crop_enc = LabelEncoder()
df_crop['region_idx'] = region_enc.fit_transform(df_crop['region'])
df_crop['crop_idx'] = crop_enc.fit_transform(df_crop['vegetable_type'])

n_regions = len(region_enc.classes_)
n_crops = len(crop_enc.classes_)

# Best crop per region-month
best_crops = df_crop.groupby(['region', 'month', 'vegetable_type']).agg({
    'yield_impact_score': 'mean', 'temperature': 'mean', 
    'rainfall': 'mean', 'humidity': 'mean'
}).reset_index()

best_crops = best_crops.loc[best_crops.groupby(['region', 'month'])['yield_impact_score'].idxmax()]
best_crops['region_idx'] = region_enc.transform(best_crops['region'])
best_crops['crop_idx'] = crop_enc.transform(best_crops['vegetable_type'])

print(f"Regions: {n_regions}, Crops: {n_crops}, Samples: {len(best_crops)}")

In [ ]:
class CropRecommender(nn.Module):
    def __init__(self, n_regions, n_crops, emb_dim=16):
        super().__init__()
        self.emb = nn.Embedding(n_regions, emb_dim)
        self.net = nn.Sequential(
            nn.Linear(emb_dim + 4, 64), nn.ReLU(), nn.BatchNorm1d(64), nn.Dropout(0.3),
            nn.Linear(64, 32), nn.ReLU(), nn.BatchNorm1d(32), nn.Dropout(0.2),
            nn.Linear(32, n_crops)
        )
    
    def forward(self, region, climate):
        x = torch.cat([self.emb(region), climate], dim=1)
        return self.net(x)

# Prepare data
X_region = best_crops['region_idx'].values
X_climate = best_crops[['temperature', 'rainfall', 'humidity', 'month']].values.astype(np.float32)
y = best_crops['crop_idx'].values

climate_scaler = StandardScaler()
X_climate_scaled = climate_scaler.fit_transform(X_climate)

X_r_train, X_r_test, X_c_train, X_c_test, y_train, y_test = train_test_split(
    X_region, X_climate_scaled, y, test_size=0.2, random_state=42
)

# Convert to tensors
X_r_train_t = torch.from_numpy(X_r_train).long().to(DEVICE)
X_c_train_t = torch.from_numpy(X_c_train).float().to(DEVICE)
y_train_t = torch.from_numpy(y_train).long().to(DEVICE)
X_r_test_t = torch.from_numpy(X_r_test).long().to(DEVICE)
X_c_test_t = torch.from_numpy(X_c_test).float().to(DEVICE)
y_test_t = torch.from_numpy(y_test).long().to(DEVICE)

print(f"Train: {len(y_train)}, Test: {len(y_test)}")

In [ ]:
# Train crop recommender
crop_model = CropRecommender(n_regions, n_crops).to(DEVICE)
opt_crop = torch.optim.Adam(crop_model.parameters(), lr=1e-3, weight_decay=1e-5)
crit_crop = nn.CrossEntropyLoss()

for epoch in range(100):
    crop_model.train()
    opt_crop.zero_grad()
    out = crop_model(X_r_train_t, X_c_train_t)
    loss = crit_crop(out, y_train_t)
    loss.backward()
    opt_crop.step()
    
    if (epoch + 1) % 20 == 0:
        crop_model.eval()
        with torch.no_grad():
            pred = crop_model(X_r_test_t, X_c_test_t).argmax(1)
            acc = (pred == y_test_t).float().mean()
        print(f'Epoch {epoch+1}: Loss={loss.item():.4f}, Acc={acc:.2%}')

## 10. Yield Prediction Model

In [ ]:
class YieldPredictor(nn.Module):
    def __init__(self, n_regions, n_crops, emb_dim=8):
        super().__init__()
        self.region_emb = nn.Embedding(n_regions, emb_dim)
        self.crop_emb = nn.Embedding(n_crops, emb_dim)
        self.net = nn.Sequential(
            nn.Linear(emb_dim * 2 + 4, 64), nn.ReLU(), nn.BatchNorm1d(64), nn.Dropout(0.3),
            nn.Linear(64, 32), nn.ReLU(), nn.Linear(32, 1)
        )
    
    def forward(self, region, crop, climate):
        x = torch.cat([self.region_emb(region), self.crop_emb(crop), climate], dim=1)
        return self.net(x)

# Prepare yield data (sample for speed)
df_yield = df_hist[['region', 'vegetable_type', 'temperature', 'rainfall', 
                    'humidity', 'yield_impact_score', 'date']].dropna()
df_yield['month'] = df_yield['date'].dt.month
df_yield['region_idx'] = region_enc.transform(df_yield['region'])
df_yield['crop_idx'] = crop_enc.transform(df_yield['vegetable_type'])
df_yield = df_yield.sample(n=min(50000, len(df_yield)), random_state=42)

X_r_y = df_yield['region_idx'].values
X_c_y = df_yield['crop_idx'].values
X_clim_y = df_yield[['temperature', 'rainfall', 'humidity', 'month']].values.astype(np.float32)
y_yield = df_yield['yield_impact_score'].values.astype(np.float32).reshape(-1, 1)

X_clim_y_scaled = climate_scaler.transform(X_clim_y)
yield_scaler = StandardScaler()
y_yield_scaled = yield_scaler.fit_transform(y_yield)

(X_r_y_train, X_r_y_test, X_c_y_train, X_c_y_test, 
 X_clim_y_train, X_clim_y_test, y_y_train, y_y_test) = train_test_split(
    X_r_y, X_c_y, X_clim_y_scaled, y_yield_scaled, test_size=0.2, random_state=42
)

print(f"Yield prediction samples: {len(y_y_train)}")

In [ ]:
# Train yield predictor
yield_model = YieldPredictor(n_regions, n_crops).to(DEVICE)
opt_yield = torch.optim.Adam(yield_model.parameters(), lr=1e-3)
crit_yield = nn.MSELoss()

# Tensors
X_r_y_t = torch.from_numpy(X_r_y_train).long().to(DEVICE)
X_c_y_t = torch.from_numpy(X_c_y_train).long().to(DEVICE)
X_clim_y_t = torch.from_numpy(X_clim_y_train).float().to(DEVICE)
y_y_t = torch.from_numpy(y_y_train).float().to(DEVICE)

for epoch in range(50):
    yield_model.train()
    idx = np.random.permutation(len(y_y_t))
    
    for start in range(0, len(idx), 256):
        batch = idx[start:start+256]
        opt_yield.zero_grad()
        out = yield_model(X_r_y_t[batch], X_c_y_t[batch], X_clim_y_t[batch])
        loss = crit_yield(out, y_y_t[batch])
        loss.backward()
        opt_yield.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}: Loss={loss.item():.4f}')

# Evaluate
yield_model.eval()
with torch.no_grad():
    pred = yield_model(
        torch.from_numpy(X_r_y_test).long().to(DEVICE),
        torch.from_numpy(X_c_y_test).long().to(DEVICE),
        torch.from_numpy(X_clim_y_test).float().to(DEVICE)
    ).cpu().numpy()
    pred_orig = yield_scaler.inverse_transform(pred)
    y_orig = yield_scaler.inverse_transform(y_y_test)
    
print(f'Yield RMSE: {np.sqrt(mean_squared_error(y_orig, pred_orig)):.4f}')

## 11. Area Allocation Optimizer

In [ ]:
from scipy.optimize import linprog

def predict_yield_single(region, crop, temp, rain, humid, month):
    yield_model.eval()
    try:
        r_idx = region_enc.transform([region])[0]
        c_idx = crop_enc.transform([crop])[0]
    except ValueError:
        return 1.0
    
    clim = climate_scaler.transform([[temp, rain, humid, month]])
    with torch.no_grad():
        pred = yield_model(
            torch.tensor([r_idx]).long().to(DEVICE),
            torch.tensor([c_idx]).long().to(DEVICE),
            torch.from_numpy(clim).float().to(DEVICE)
        ).cpu().numpy()
    return float(yield_scaler.inverse_transform(pred)[0, 0])

class AreaOptimizer:
    def __init__(self, crops, region, total_area):
        self.crops = crops
        self.region = region
        self.total_area = total_area
    
    def get_prices(self):
        return np.array([
            df_hist[df_hist['vegetable_type'] == c]['vegetable_price'].mean() or 100
            for c in self.crops
        ])
    
    def optimize(self, temp, rain, humid, month, min_div=0.05):
        yields = np.array([predict_yield_single(self.region, c, temp, rain, humid, month) 
                          for c in self.crops])
        yields = np.maximum(yields, 0.1)
        prices = self.get_prices()
        
        c = -(yields * prices)
        A_eq = np.ones((1, len(self.crops)))
        b_eq = np.array([self.total_area])
        bounds = [(min_div * self.total_area, 
                   (1 - min_div * (len(self.crops) - 1)) * self.total_area) 
                  for _ in self.crops]
        
        result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')
        alloc = result.x if result.success else np.full(len(self.crops), self.total_area / len(self.crops))
        
        return {
            'crops': self.crops,
            'allocation_ha': alloc.round(1).tolist(),
            'allocation_pct': (alloc / self.total_area * 100).round(1).tolist(),
            'expected_revenue': float(np.sum(alloc * yields * prices))
        }

In [ ]:
# Test optimizer
top_crops = df_hist['vegetable_type'].value_counts().head(5).index.tolist()
opt = AreaOptimizer(top_crops, 'Anuradhapura', 100)

for name, params in [
    ('Maha', {'temp': 28, 'rain': 250, 'humid': 85, 'month': 11}),
    ('Yala', {'temp': 33, 'rain': 50, 'humid': 60, 'month': 5})
]:
    result = opt.optimize(**params)
    print(f"\n{name} Season:")
    for c, a, p in zip(result['crops'], result['allocation_ha'], result['allocation_pct']):
        print(f"  {c}: {a} ha ({p}%)")
    print(f"  Revenue: LKR {result['expected_revenue']:,.0f}")

## 11.1 Price Forecasting (LSTM)

In [ ]:
# =============================================================================
# Price LSTM Model Definition
# =============================================================================

class PriceLSTM(nn.Module):
    def __init__(self, input_size, hidden=64, layers=2, horizon=6):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden, layers, batch_first=True, dropout=0.2)
        self.fc = nn.Sequential(nn.Linear(hidden, 32), nn.ReLU(), nn.Linear(32, horizon))
    
    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

# Prepare data
WINDOW, HORIZON = 30, 6
feature_cols = ['price', 'production', 'exchange_rate', 'fuel_price']
features = df_rice[feature_cols].values.astype(np.float32)

price_scaler = MinMaxScaler()
features_scaled = price_scaler.fit_transform(features)

def make_sequences(data, window, horizon):
    X, Y = [], []
    for i in range(len(data) - window - horizon + 1):
        X.append(data[i:i+window])
        Y.append(data[i+window:i+window+horizon, 0])
    return np.array(X), np.array(Y)

X_price, Y_price = make_sequences(features_scaled, WINDOW, HORIZON)
split = int(len(X_price) * 0.8)

X_train_p = torch.from_numpy(X_price[:split]).float().to(DEVICE)
Y_train_p = torch.from_numpy(Y_price[:split]).float().to(DEVICE)
X_test_p = torch.from_numpy(X_price[split:]).float().to(DEVICE)
Y_test_p = torch.from_numpy(Y_price[split:]).float().to(DEVICE)

print(f"Sequences: X={X_price.shape}, Train={len(X_train_p)}, Test={len(X_test_p)}")

## 11.2 Advanced Area Allocation Optimizer

In [ ]:
from scipy.optimize import linprog

def predict_yield_for_crop(region, crop, temperature, rainfall, humidity, month):
    yield_model.eval()
    try:
        r_idx = region_enc.transform([region])[0]
        c_idx = crop_enc.transform([crop])[0]
    except ValueError:
        return 1.0
    
    climate = climate_scaler.transform([[temperature, rainfall, humidity, month]])
    with torch.no_grad():
        pred = yield_model(
            torch.tensor([r_idx]).long().to(DEVICE),
            torch.tensor([c_idx]).long().to(DEVICE),
            torch.from_numpy(climate).float().to(DEVICE)
        )
        return float(yield_scaler.inverse_transform(pred.cpu().numpy())[0, 0])

class AreaOptimizer:
    def __init__(self, crops, region, total_area):
        self.crops = crops
        self.region = region
        self.total_area = total_area
        self.n_crops = len(crops)
    
    def get_yields(self, temp, rain, hum, month):
        return np.array([max(predict_yield_for_crop(self.region, c, temp, rain, hum, month), 0.1) for c in self.crops])
    
    def get_prices(self):
        prices = []
        for crop in self.crops:
            data = df_hist[df_hist['vegetable_type'] == crop]
            prices.append(data['vegetable_price'].mean() if len(data) > 0 else 100)
        return np.array(prices)
    
    def optimize(self, temp, rain, hum, month, min_div=0.05):
        yields = self.get_yields(temp, rain, hum, month)
        prices = self.get_prices()
        
        c = -(yields * prices)
        A_eq = np.ones((1, self.n_crops))
        b_eq = np.array([self.total_area])
        bounds = [(min_div * self.total_area, (1 - min_div * (self.n_crops - 1)) * self.total_area)] * self.n_crops
        
        result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')
        alloc = result.x if result.success else np.full(self.n_crops, self.total_area / self.n_crops)
        
        return {
            'crops': self.crops,
            'allocations_ha': alloc.tolist(),
            'allocation_pct': (alloc / self.total_area * 100).tolist(),
            'expected_yields': yields.tolist(),
            'expected_prices': prices.tolist(),
            'total_expected_revenue': np.sum(alloc * yields * prices),
            'success': result.success if hasattr(result, 'success') else True
        }

print("AreaOptimizer defined")

In [ ]:
# Test area optimizer
available_crops = df_hist['vegetable_type'].value_counts().head(5).index.tolist()
print(f"Crops: {available_crops}")

optimizer = AreaOptimizer(crops=available_crops, region='Anuradhapura', total_area=100)

scenarios = [
    ("Wet Season", 28, 250, 85, 11),
    ("Dry Season", 33, 50, 60, 5),
    ("Inter-monsoon", 30, 150, 75, 3),
]

for name, temp, rain, hum, month in scenarios:
    result = optimizer.optimize(temp, rain, hum, month)
    print(f"\n{name}: {temp}C, {rain}mm, {hum}%")
    for c, a, p in zip(result['crops'], result['allocations_ha'], result['allocation_pct']):
        print(f"  {c}: {a:.1f}ha ({p:.1f}%)")
    print(f"  Revenue: LKR {result['total_expected_revenue']:,.0f}")

## 12. Model Generalization & Cross-Validation Analysis

This section evaluates model generalization capabilities through:
- K-Fold Cross-Validation for standard models
- Time Series Cross-Validation for temporal models
- Learning curve analysis
- Performance stability assessment

In [ ]:
# =============================================================================
# 11.1 K-Fold Cross-Validation for Yield Prediction
# =============================================================================

def kfold_evaluation(model_class, X, y, n_splits=5, epochs=50, device=DEVICE):
    """Perform K-Fold CV and return fold-wise metrics."""
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    fold_metrics = {'train_loss': [], 'val_loss': [], 'r2': [], 'mae': []}
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        X_train_fold, X_val_fold = X[train_idx], X[val_idx]
        y_train_fold, y_val_fold = y[train_idx], y[val_idx]
        
        # Convert to tensors
        X_train_t = torch.FloatTensor(X_train_fold).to(device)
        y_train_t = torch.FloatTensor(y_train_fold).reshape(-1, 1).to(device)
        X_val_t = torch.FloatTensor(X_val_fold).to(device)
        y_val_t = torch.FloatTensor(y_val_fold).reshape(-1, 1).to(device)
        
        # Initialize model
        model = model_class(X.shape[1]).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.MSELoss()
        
        # Train
        model.train()
        for _ in range(epochs):
            optimizer.zero_grad()
            pred = model(X_train_t)
            loss = criterion(pred, y_train_t)
            loss.backward()
            optimizer.step()
        
        # Evaluate
        model.eval()
        with torch.no_grad():
            train_pred = model(X_train_t).cpu().numpy()
            val_pred = model(X_val_t).cpu().numpy()
            
        fold_metrics['train_loss'].append(mean_squared_error(y_train_fold, train_pred))
        fold_metrics['val_loss'].append(mean_squared_error(y_val_fold, val_pred))
        fold_metrics['r2'].append(r2_score(y_val_fold, val_pred))
        fold_metrics['mae'].append(mean_absolute_error(y_val_fold, val_pred))
    
    return fold_metrics

# Simple MLP for CV testing
class SimpleMLP(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )
    def forward(self, x):
        return self.net(x)

# Prepare data for CV
X_cv = X_clim_y_scaled if 'X_clim_y_scaled' in dir() else X_climate_scaled
y_cv = y_yield_scaled if 'y_yield_scaled' in dir() else y_yield

print("K-FOLD CROSS-VALIDATION: Yield Predictor")
print("=" * 60)
cv_results = kfold_evaluation(SimpleMLP, X_cv, y_cv, n_splits=5, epochs=100)

for metric, values in cv_results.items():
    print(f"{metric:12}: Mean = {np.mean(values):.4f} ± {np.std(values):.4f}")

In [ ]:
# =============================================================================
# 11.2 Time Series Cross-Validation for Price LSTM
# =============================================================================

def timeseries_cv(X, Y, n_splits=3, window=WINDOW):
    """Time series cross-validation with expanding window."""
    tscv = TimeSeriesSplit(n_splits=n_splits)
    ts_metrics = {'fold': [], 'train_size': [], 'test_size': [], 'rmse': [], 'mae': []}
    
    for fold, (train_idx, test_idx) in enumerate(tscv.split(X)):
        X_train, X_test = X[train_idx], X[test_idx]
        Y_train, Y_test = Y[train_idx], Y[test_idx]
        
        # Use simple linear regression as baseline
        X_train_flat = X_train.reshape(X_train.shape[0], -1)
        X_test_flat = X_test.reshape(X_test.shape[0], -1)
        Y_train_flat = Y_train.reshape(Y_train.shape[0], -1)
        Y_test_flat = Y_test.reshape(Y_test.shape[0], -1)
        
        lr = LinearRegression()
        lr.fit(X_train_flat, Y_train_flat)
        pred = lr.predict(X_test_flat)
        
        ts_metrics['fold'].append(fold + 1)
        ts_metrics['train_size'].append(len(train_idx))
        ts_metrics['test_size'].append(len(test_idx))
        ts_metrics['rmse'].append(np.sqrt(mean_squared_error(Y_test_flat, pred)))
        ts_metrics['mae'].append(mean_absolute_error(Y_test_flat, pred))
    
    return pd.DataFrame(ts_metrics)

print("TIME SERIES CROSS-VALIDATION: Price Forecasting")
print("=" * 60)
ts_cv_results = timeseries_cv(X_rice, Y_rice, n_splits=4)
print(ts_cv_results.to_string(index=False))
print(f"\nMean RMSE: {ts_cv_results['rmse'].mean():.4f} ± {ts_cv_results['rmse'].std():.4f}")

In [ ]:
# =============================================================================
# 11.3 Model Performance Visualization
# =============================================================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Model Generalization Analysis', fontsize=14, fontweight='bold')

# 1. K-Fold CV Results
ax = axes[0, 0]
fold_nums = range(1, 6)
ax.bar(fold_nums, cv_results['val_loss'], alpha=0.7, label='Validation Loss', color='#3498db')
ax.axhline(np.mean(cv_results['val_loss']), color='red', linestyle='--', label=f"Mean: {np.mean(cv_results['val_loss']):.4f}")
ax.fill_between(fold_nums, 
                np.mean(cv_results['val_loss']) - np.std(cv_results['val_loss']),
                np.mean(cv_results['val_loss']) + np.std(cv_results['val_loss']),
                alpha=0.2, color='red', label='±1 Std')
ax.set_xlabel('Fold')
ax.set_ylabel('MSE Loss')
ax.set_title('K-Fold CV: Yield Predictor')
ax.legend()
ax.set_xticks(fold_nums)

# 2. Time Series CV Results
ax = axes[0, 1]
x_ts = range(len(ts_cv_results))
ax.plot(ts_cv_results['train_size'], ts_cv_results['rmse'], 'o-', label='RMSE vs Train Size', markersize=8)
ax.set_xlabel('Training Set Size')
ax.set_ylabel('RMSE')
ax.set_title('Learning Curve: Price Forecasting')
for i, (train, rmse) in enumerate(zip(ts_cv_results['train_size'], ts_cv_results['rmse'])):
    ax.annotate(f'Fold {i+1}', (train, rmse), textcoords='offset points', xytext=(5, 5), fontsize=8)

# 3. R² Distribution
ax = axes[1, 0]
r2_values = cv_results['r2']
ax.hist(r2_values, bins=5, edgecolor='black', alpha=0.7, color='#2ecc71')
ax.axvline(np.mean(r2_values), color='red', linestyle='--', label=f"Mean R²: {np.mean(r2_values):.3f}")
ax.set_xlabel('R² Score')
ax.set_ylabel('Frequency')
ax.set_title('R² Score Distribution (K-Fold)')
ax.legend()

# 4. Train vs Validation Loss Comparison
ax = axes[1, 1]
width = 0.35
x = np.arange(len(fold_nums))
ax.bar(x - width/2, cv_results['train_loss'], width, label='Train', alpha=0.7, color='#3498db')
ax.bar(x + width/2, cv_results['val_loss'], width, label='Validation', alpha=0.7, color='#e74c3c')
ax.set_xlabel('Fold')
ax.set_ylabel('MSE Loss')
ax.set_title('Train vs Validation Loss')
ax.set_xticks(x)
ax.set_xticklabels([f'Fold {i}' for i in fold_nums])
ax.legend()

# Calculate generalization gap
gen_gap = np.mean(np.array(cv_results['val_loss']) - np.array(cv_results['train_loss']))
ax.text(0.95, 0.95, f'Generalization Gap: {gen_gap:.4f}', transform=ax.transAxes,
        fontsize=9, ha='right', va='top', bbox=dict(boxstyle='round', facecolor='wheat'))

plt.tight_layout()
plt.show()

print("\nGENERALIZATION SUMMARY")
print("=" * 60)
print(f"Yield Predictor - Mean Validation MSE: {np.mean(cv_results['val_loss']):.4f}")
print(f"Yield Predictor - Mean R²: {np.mean(cv_results['r2']):.4f}")
print(f"Price LSTM Baseline - Mean RMSE: {ts_cv_results['rmse'].mean():.4f}")
print(f"Generalization Gap: {gen_gap:.4f} {'(Low - Good)' if abs(gen_gap) < 0.1 else '(High - Overfitting risk)'}")

In [ ]:
# =============================================================================
# 11.4 Baseline Model Comparison
# =============================================================================

print("BASELINE MODEL COMPARISON")
print("=" * 60)

# Prepare data
X_baseline = X_cv
y_baseline = y_cv

# Split
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(
    X_baseline, y_baseline, test_size=0.2, random_state=RANDOM_SEED
)

# Models to compare
baseline_results = []

# 1. Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train_b, y_train_b)
pred_lr = lr_model.predict(X_test_b)
baseline_results.append({
    'Model': 'Linear Regression',
    'R²': r2_score(y_test_b, pred_lr),
    'RMSE': np.sqrt(mean_squared_error(y_test_b, pred_lr)),
    'MAE': mean_absolute_error(y_test_b, pred_lr)
})

# 2. Random Forest
rf = RandomForestRegressor(n_estimators=50, random_state=RANDOM_SEED, n_jobs=-1)
rf.fit(X_train_b, y_train_b)
pred_rf = rf.predict(X_test_b)
baseline_results.append({
    'Model': 'Random Forest',
    'R²': r2_score(y_test_b, pred_rf),
    'RMSE': np.sqrt(mean_squared_error(y_test_b, pred_rf)),
    'MAE': mean_absolute_error(y_test_b, pred_rf)
})

# 3. Neural Network (Simple MLP)
class BaselineMLP(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
    def forward(self, x):
        return self.net(x)

mlp_baseline = BaselineMLP(X_train_b.shape[1]).to(DEVICE)
opt_mlp = torch.optim.Adam(mlp_baseline.parameters(), lr=0.01)
crit_mlp = nn.MSELoss()

X_train_t = torch.FloatTensor(X_train_b).to(DEVICE)
y_train_t = torch.FloatTensor(y_train_b).reshape(-1, 1).to(DEVICE)
X_test_t = torch.FloatTensor(X_test_b).to(DEVICE)

mlp_baseline.train()
for _ in range(100):
    opt_mlp.zero_grad()
    out = mlp_baseline(X_train_t)
    loss = crit_mlp(out, y_train_t)
    loss.backward()
    opt_mlp.step()

mlp_baseline.eval()
with torch.no_grad():
    pred_mlp = mlp_baseline(X_test_t).cpu().numpy().flatten()

baseline_results.append({
    'Model': 'MLP Neural Network',
    'R²': r2_score(y_test_b, pred_mlp),
    'RMSE': np.sqrt(mean_squared_error(y_test_b, pred_mlp)),
    'MAE': mean_absolute_error(y_test_b, pred_mlp)
})

# Display results
baseline_df = pd.DataFrame(baseline_results)
print(baseline_df.to_string(index=False))

# Visualize comparison
fig, ax = plt.subplots(figsize=(10, 5))
x = np.arange(len(baseline_df))
width = 0.25

ax.bar(x - width, baseline_df['R²'], width, label='R²', alpha=0.8)
ax.bar(x, baseline_df['RMSE'], width, label='RMSE', alpha=0.8)
ax.bar(x + width, baseline_df['MAE'], width, label='MAE', alpha=0.8)

ax.set_ylabel('Score')
ax.set_title('Model Performance Comparison')
ax.set_xticks(x)
ax.set_xticklabels(baseline_df['Model'])
ax.legend()
plt.tight_layout()
plt.show()

## 13. Multi-Crop Price Forecasting LSTM

In [ ]:
class MultiCropPriceLSTM(nn.Module):
    def __init__(self, input_size, hidden=64, layers=2, dropout=0.2, horizon=7):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden, layers, batch_first=True, dropout=dropout if layers > 1 else 0)
        self.fc = nn.Sequential(nn.Linear(hidden, 32), nn.ReLU(), nn.Linear(32, horizon))
    
    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

# Use existing data from earlier cell
df_rice_clean = df_rice.copy()
price_feature_scaler = MinMaxScaler()
price_scaled = price_feature_scaler.fit_transform(features)

def make_seqs(data, target_col, window, horizon):
    X, Y = [], []
    for i in range(len(data) - window - horizon + 1):
        X.append(data[i:i+window])
        Y.append(data[i+window:i+window+horizon, target_col])
    return np.array(X), np.array(Y)

X_rice, Y_rice = make_seqs(price_scaled, 0, WINDOW, HORIZON)
split_rice = int(len(X_rice) * 0.8)

X_rice_train_t = torch.from_numpy(X_rice[:split_rice]).float().to(DEVICE)
Y_rice_train_t = torch.from_numpy(Y_rice[:split_rice]).float().to(DEVICE)
X_rice_test_t = torch.from_numpy(X_rice[split_rice:]).float().to(DEVICE)
Y_rice_test_t = torch.from_numpy(Y_rice[split_rice:]).float().to(DEVICE)

print(f"Rice forecast: X={X_rice.shape}, Train={len(X_rice_train_t)}, Test={len(X_rice_test_t)}")

In [ ]:
# Train price forecaster
rice_price_model = MultiCropPriceLSTM(len(feature_cols), horizon=HORIZON).to(DEVICE)
opt_price = torch.optim.Adam(rice_price_model.parameters(), lr=1e-3)
crit_price = nn.MSELoss()

for epoch in range(1, 101):
    rice_price_model.train()
    idx = np.random.permutation(len(X_rice_train_t))
    
    for start in range(0, len(idx), 16):
        batch = idx[start:start+16]
        opt_price.zero_grad()
        loss = crit_price(rice_price_model(X_rice_train_t[batch]), Y_rice_train_t[batch])
        loss.backward()
        opt_price.step()
    
    if epoch % 20 == 0:
        rice_price_model.eval()
        with torch.no_grad():
            test_loss = crit_price(rice_price_model(X_rice_test_t), Y_rice_test_t)
        print(f'Epoch {epoch}: Train={loss.item():.5f}, Test={test_loss:.5f}')

# Evaluate
rice_price_model.eval()
with torch.no_grad():
    test_pred = rice_price_model(X_rice_test_t).cpu().numpy()
    test_actual = Y_rice_test_t.cpu().numpy()

def inv_price(x):
    dummy = np.zeros((len(x), len(feature_cols)))
    dummy[:, 0] = x
    return price_feature_scaler.inverse_transform(dummy)[:, 0]

pred_1m = inv_price(test_pred[:, 0])
actual_1m = inv_price(test_actual[:, 0])
rmse_price = np.sqrt(mean_squared_error(actual_1m, pred_1m))
print(f'\nPrice Forecast (1-month) - RMSE: {rmse_price:.2f} LKR')

plt.figure(figsize=(12, 4))
plt.plot(actual_1m, label='Actual', alpha=0.8)
plt.plot(pred_1m, label='Predicted', alpha=0.8)
plt.xlabel('Sample'); plt.ylabel('Price (LKR)'); plt.title('Rice Price Forecast')
plt.legend(); plt.tight_layout(); plt.show()

## 14. Model Export

In [ ]:
import pickle, json

MODELS_DIR = Path('../app/models')
MODELS_DIR.mkdir(parents=True, exist_ok=True)

# Save PyTorch models
torch.save({'model_state_dict': crop_model.state_dict(), 'n_regions': n_regions, 'n_crops': n_crops, 'embedding_dim': 16}, MODELS_DIR / 'crop_recommender.pt')
torch.save({'model_state_dict': yield_model.state_dict(), 'n_regions': n_regions, 'n_crops': n_crops, 'embedding_dim': 8}, MODELS_DIR / 'yield_predictor.pt')
torch.save({'model_state_dict': rice_price_model.state_dict(), 'input_size': len(feature_cols), 'hidden_size': 64, 'num_layers': 2, 'horizon': HORIZON, 'window': WINDOW}, MODELS_DIR / 'rice_price_lstm.pt')

print("Saved: crop_recommender.pt, yield_predictor.pt, rice_price_lstm.pt")

In [ ]:
# Save encoders and scalers
for name, obj in [('region_encoder', region_enc), ('crop_encoder', crop_enc), 
                  ('climate_scaler', climate_scaler), ('yield_scaler', yield_scaler), 
                  ('price_feature_scaler', price_feature_scaler)]:
    with open(MODELS_DIR / f'{name}.pkl', 'wb') as f:
        pickle.dump(obj, f)

metadata = {
    'regions': region_enc.classes_.tolist(),
    'crops': crop_enc.classes_.tolist(),
    'climate_features': ['temperature', 'rainfall', 'humidity', 'month'],
    'price_features': feature_cols,
    'forecast_window': WINDOW,
    'forecast_horizon': HORIZON,
    'created_at': pd.Timestamp.now().isoformat()
}
with open(MODELS_DIR / 'metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"All artifacts saved to: {MODELS_DIR.resolve()}")

## 15. Service Interface

In [ ]:
def recommend_crop(region, temp, rain, hum, month, top_k=5):
    crop_model.eval()
    try:
        r_idx = region_enc.transform([region])[0]
    except ValueError:
        return []
    
    climate = climate_scaler.transform([[temp, rain, hum, month]])
    with torch.no_grad():
        logits = crop_model(
            torch.tensor([r_idx]).long().to(DEVICE),
            torch.from_numpy(climate).float().to(DEVICE)
        )
        probs = torch.softmax(logits, dim=1)[0].cpu().numpy()
    
    top_idx = np.argsort(probs)[-top_k:][::-1]
    return [(crop_enc.classes_[i], float(probs[i])) for i in top_idx]

class ACAOService:
    def __init__(self):
        self.device = DEVICE
        self.crop_model = crop_model
        self.yield_model = yield_model
        self.price_model = rice_price_model
        self.region_enc = region_enc
        self.crop_enc = crop_enc
        self.climate_scaler = climate_scaler
        self.yield_scaler = yield_scaler
        self.price_scaler = price_feature_scaler
        self.regions = region_enc.classes_.tolist()
        self.crops = crop_enc.classes_.tolist()
    
    def get_crop_recommendations(self, region, temp, rain, hum, month, top_k=5):
        return recommend_crop(region, temp, rain, hum, month, top_k)
    
    def predict_yield(self, region, crop, temp, rain, hum, month):
        return predict_yield_for_crop(region, crop, temp, rain, hum, month)
    
    def optimize_area(self, region, crops, total_area, temp, rain, hum, month, min_div=0.05):
        opt = AreaOptimizer(crops=crops, region=region, total_area=total_area)
        return opt.optimize(temp, rain, hum, month, min_div)
    
    def get_regions(self): return self.regions
    def get_crops(self): return self.crops
    
    def get_regional_summary(self, region):
        if region in regional.index:
            d = regional.loc[region].to_dict()
            return {'region': region, 'avg_temp': d.get('temperature', 0), 
                    'avg_rain': d.get('rainfall', 0), 'avg_yield': d.get('yield_impact_score', 0)}
        return {'region': region, 'error': 'Not found'}

acao_service = ACAOService()
print(f"ACA-O Service: {len(acao_service.regions)} regions, {len(acao_service.crops)} crops")

In [ ]:
# Demo workflow
demo = {'region': 'Anuradhapura', 'month': 10, 'temp': 29, 'rain': 180, 'hum': 78}
print(f"Demo: {demo['region']}, Maha Season\nClimate: {demo['temp']}C, {demo['rain']}mm, {demo['hum']}%")

# Regional summary
summary = acao_service.get_regional_summary(demo['region'])
print(f"\nHistorical: temp={summary.get('avg_temp', 'N/A')}, rain={summary.get('avg_rain', 'N/A')}")

# Crop recommendations
print("\nTop 5 Crops:")
recs = acao_service.get_crop_recommendations(demo['region'], demo['temp'], demo['rain'], demo['hum'], demo['month'], 5)
for i, (c, conf) in enumerate(recs, 1):
    print(f"  {i}. {c}: {conf:.1%}")

# Yield predictions
print("\nYield Predictions:")
top_crops = [c for c, _ in recs[:5]]
for c in top_crops:
    y = acao_service.predict_yield(demo['region'], c, demo['temp'], demo['rain'], demo['hum'], demo['month'])
    print(f"  {c}: {y:.2f}")

# Area optimization
print("\nOptimal Allocation (100 ha):")
opt = acao_service.optimize_area(demo['region'], top_crops, 100, demo['temp'], demo['rain'], demo['hum'], demo['month'])
for c, a, p in zip(opt['crops'], opt['allocations_ha'], opt['allocation_pct']):
    print(f"  {c}: {a:.1f}ha ({p:.1f}%)")
print(f"\nExpected Revenue: LKR {opt['total_expected_revenue']:,.0f}")

## 16. Dashboard

In [ ]:
fig = plt.figure(figsize=(16, 10))

ax1 = fig.add_subplot(2, 3, 1)
top_reg = regional.nlargest(10, 'yield_impact_score')
ax1.barh(top_reg.index, top_reg['yield_impact_score'], color='forestgreen')
ax1.set_xlabel('Yield Score'); ax1.set_title('Top Regions by Yield'); ax1.invert_yaxis()

ax2 = fig.add_subplot(2, 3, 2)
sc = ax2.scatter(regional['temperature'], regional['yield_impact_score'],
                 c=regional['rainfall'], cmap='Blues', s=80, alpha=0.7)
ax2.set_xlabel('Temp (C)'); ax2.set_ylabel('Yield Score'); ax2.set_title('Climate vs Yield')
plt.colorbar(sc, ax=ax2, label='Rain (mm)')

ax3 = fig.add_subplot(2, 3, 3)
crops_dist = df_hist['vegetable_type'].value_counts().head(8)
ax3.pie(crops_dist.values, labels=crops_dist.index, autopct='%1.0f%%')
ax3.set_title('Crop Distribution')

ax4 = fig.add_subplot(2, 3, 4)
df_hist['month'] = df_hist['date'].dt.month
monthly = df_hist.groupby('month')['vegetable_price'].mean()
ax4.plot(monthly.index, monthly.values, 'o-', lw=2)
ax4.set_xlabel('Month'); ax4.set_ylabel('Price (LKR)'); ax4.set_title('Seasonal Prices')

ax5 = fig.add_subplot(2, 3, 5)
ax5.plot(df_rice['date'], df_rice['price'], lw=0.8)
ax5.set_xlabel('Date'); ax5.set_ylabel('Price (LKR)'); ax5.set_title('Rice Price History')

ax6 = fig.add_subplot(2, 3, 6)
models = ['Crop Rec', 'Yield', 'Price']
scores = [acc.item() * 100, max(0, 100 - 0.1525 * 100), max(0, 100 - rmse_price / 10)]
ax6.bar(models, scores, color=['#2ecc71', '#3498db', '#9b59b6'])
ax6.set_ylabel('Score'); ax6.set_title('Model Performance'); ax6.set_ylim(0, 100)

plt.suptitle('ACA-O Sri Lanka Dashboard', fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout(); plt.show()

## 17. Conclusion & Summary

### System Overview
The ACA-O (Adaptive Crop & Area Optimization) system integrates multiple ML models for agricultural decision support in Sri Lanka.

### Dataset Summary
| Dataset | Records | Purpose |
|---------|---------|---------|
| Paddy Statistics | 28 | Regional crop-yield baselines |
| Economy (Rice) | 61 | Economic indicators, production trends |
| Vegetable Prices | 39 | Current market pricing |
| Historical Vegetables | 130,000 | Price pattern learning |
| Rice Time Series | 324 | Monthly price sequences |
| Climate Data | 1,800 | Weather-yield correlations |

### Model Performance
| Model | Architecture | Metric | Value |
|-------|--------------|--------|-------|
| CropRecommender | Region Embedding + MLP | Accuracy | ~36% |
| YieldPredictor | Dual Embedding + MLP | RMSE | ~0.48 |
| PriceLSTM | LSTM (2 layers) | RMSE | ~0.06 |
| AreaOptimizer | Linear Programming | Revenue | Optimized |

### Key Insights
1. **Data Quality**: Historical vegetable dataset provides most training value
2. **Regional Patterns**: 25 Sri Lankan districts show distinct climate-yield correlations
3. **Seasonality**: Maha and Yala seasons significantly impact optimal crop selection
4. **Price Volatility**: LSTM captures multi-horizon price patterns effectively

### Next Steps
- Integrate real-time weather API data
- Add more official government agricultural statistics
- Implement ensemble methods for improved accuracy
- Deploy as microservice in the irrigation system